In [1]:
import numpy as np
import tensorflow as tf
import random
import sys
import eli5

import d2d
import importlib
importlib.reload(d2d)
import pandas as pd
import os

2022-11-16 10:27:10.562987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 10:27:10.668533: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-16 10:27:10.700009: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 10:27:11.200690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:


SEED = 1
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

file = "/data/fast0/datasets/Rhone_data_continuous_highpass.h5"

filt = 'Highpass'

history = {}
val_performance = {}
performance = {}

ww = 200
bs = 32

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

input_columns = list(np.arange(0, 2308, 1))


linear_model, lstm_model, dnn_model, df_all_chan, das_data_all, f  = d2d.import_data(filename = file, input_columns = input_columns)

da = df_all_chan.to_numpy()

multi_step_window_shuffled = d2d.WindowGenerator(df_all_chan,
                                                input_width=ww,
                                                label_width=1, 
                                                shift=0,
                                                label_columns=['Discharge'],
                                                input_columns=input_columns,
                                                shuffle=True,
                                                batch_size=bs)

history['lstm_whole_cable'] = d2d.compile_and_fit(lstm_model, multi_step_window_shuffled, learning_rate = 0.001)

val_performance['lstm_whole_cable'] = lstm_model.evaluate(multi_step_window_shuffled.val)
performance['lstm_whole_cable'] = lstm_model.evaluate(multi_step_window_shuffled.test, verbose=0)

Keys: <KeysViewHDF5 ['DAS Data', 'Discharge', 'Times']>


2022-11-16 10:27:12.727099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-11-16 10:27:12.727122: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-16 10:27:12.727420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
prediction_set = d2d.WindowGenerator(df_all_chan,
                                    input_width=ww,
                                    label_width=1, 
                                    shift=0,
                                    label_columns=['Discharge'],
                                    input_columns=input_columns,
                                    shuffle=False,
                                    batch_size=bs)

In [ ]:
predictions = lstm_model.predict(prediction_set.train).squeeze()

predictions_real = (predictions * prediction_set.dis_std) + prediction_set.dis_mean

In [ ]:
real_dis = []

for i in prediction_set.train.as_numpy_iterator():
    real_dis.append(i[-1])
    
real_dis = np.asarray(real_dis)
real_dis = real_dis.reshape(real_dis.shape[0] * real_dis.shape[1])

real_dis = (real_dis * prediction_set.dis_std) + prediction_set.dis_mean

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

fig, ax = plt.subplots(figsize = [15,10])
fig.patch.set_facecolor('white')

ax.plot(predictions_real, label = 'Predicted', lw = 3, color = 'brown', linestyle = '--')
ax.plot(real_dis, label = 'Observed', lw = 3, color = 'steelblue')

ax.set_ylabel('Discharge (m$^3$/s)', size=20)
ax.set_xlabel('Time Step', size=20)

ax.tick_params(labelsize=16)

ax.legend(prop={'size': 20})

#fig.savefig('discharge_train_timeseries.eps', dpi=1200)

plt.show()